# Using PyTorch

In [7]:
import torch 
import torch.nn as nn 
import numpy as np 
import torchinfo 
from torchinfo import summary 
from tqdm import tqdm
import math

def equation(input:torch.Tensor): # z = 5 * a^2 + 3 * b^2
    return torch.matmul(torch.mul(input, input), torch.tensor([5, 3], dtype=torch.float32))

num_of_data = 100000
inputs = torch.randn(num_of_data, 2)
outputs = equation(inputs)

class SimpleModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 10)
        self.fc2 = nn.Linear(10, 1)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        return x 

device = torch.device('mps')
split = int(inputs.shape[0] * 0.7)
X_train, y_train, X_test, y_test = inputs[:split].to(device), outputs[:split].to(device), inputs[split:].to(device), outputs[split:].to(device)
print(f"Shape of X_train: {X_train.shape}, y_train: {y_train.shape}, X_test: {X_test.shape}, y_test = {y_test.shape}")

model = SimpleModel().to(device)
summary(model) 

Shape of X_train: torch.Size([70000, 2]), y_train: torch.Size([70000]), X_test: torch.Size([30000, 2]), y_test = torch.Size([30000])


Layer (type:depth-idx)                   Param #
SimpleModel                              --
├─Linear: 1-1                            30
├─Linear: 1-2                            11
├─Sigmoid: 1-3                           --
Total params: 41
Trainable params: 41
Non-trainable params: 0

In [8]:
lr = 0.01
n_epochs = 10
optimizer = torch.optim.SGD(model.parameters(), lr = lr)
criterion = nn.MSELoss()
batch_size = 100
log_interval = 1

for epoch in range(n_epochs):
    print(f"====Epoch {epoch}====")

    for layer in model.children():
        print(layer.state_dict())
        
    training_loss = 0
    training_count = 0
    for batch_end_idx in tqdm(range(batch_size, X_train.shape[0], batch_size)):
        optimizer.zero_grad()
        out = model(X_train[:batch_end_idx])
        loss = criterion(out, y_train[:batch_end_idx].view(-1,1))
        loss.backward()
        optimizer.step()
        training_loss += loss.item()
        training_count += 1

    if epoch % log_interval == 0:
        model.eval()
        validation_loss = 0
        validation_count = 0
        for batch_end_idx in range(batch_size, X_test.shape[0], batch_size):
            out = model(X_test[:batch_end_idx])
            loss = criterion(out, y_test[:batch_end_idx].view(-1,1))
            validation_loss += loss.item()
            validation_count += 1
        print(f"Training Loss: {training_loss / training_count} | Validation Loss: {validation_loss / validation_count}")
        model.train()

====Epoch 0====
OrderedDict([('weight', tensor([[-0.4984, -0.5953],
        [ 0.3925,  0.0803],
        [ 0.5303,  0.6326],
        [ 0.3085, -0.0761],
        [-0.5978,  0.2972],
        [-0.0843, -0.2365],
        [ 0.5423, -0.6976],
        [-0.1007,  0.5704],
        [-0.3974,  0.5895],
        [-0.6049, -0.5847]], device='mps:0')), ('bias', tensor([-0.5867, -0.5186, -0.4147,  0.1995, -0.3381, -0.1858, -0.5737, -0.2141,
        -0.5028, -0.5999], device='mps:0'))])
OrderedDict([('weight', tensor([[ 0.0915,  0.3023,  0.1852,  0.2384, -0.1829,  0.2571,  0.0949,  0.2068,
         -0.0131,  0.2467]], device='mps:0')), ('bias', tensor([0.3007], device='mps:0'))])
OrderedDict()


100%|██████████| 699/699 [00:11<00:00, 62.51it/s] 


Training Loss: 45.71417453804071 | Validation Loss: 21.09571307000508
====Epoch 1====
OrderedDict([('weight', tensor([[-2.4222, -1.7161],
        [ 3.0377, -0.1667],
        [ 2.9845,  1.0161],
        [ 0.0391, -0.0780],
        [-2.5859,  0.7425],
        [ 0.0514, -0.3375],
        [ 2.7544, -1.7347],
        [ 0.3097,  2.2876],
        [-2.3251,  1.6415],
        [-2.9123, -1.0547]], device='mps:0')), ('bias', tensor([-3.4998, -3.7009, -3.7807,  0.1959, -3.3317, -0.2864, -3.7640, -2.3982,
        -3.3997, -3.8782], device='mps:0'))])
OrderedDict([('weight', tensor([[5.1687, 5.7475, 5.9135, 0.7232, 4.4631, 0.7141, 6.0694, 3.0183, 4.7189,
         5.8861]], device='mps:0')), ('bias', tensor([1.1393], device='mps:0'))])
OrderedDict()


100%|██████████| 699/699 [00:10<00:00, 64.63it/s] 


Training Loss: 12.924530703281299 | Validation Loss: 7.860880938262047
====Epoch 2====
OrderedDict([('weight', tensor([[-1.8588, -2.8719],
        [ 3.4279, -0.2854],
        [ 3.2102,  1.5446],
        [ 0.1000, -0.1051],
        [-3.1487,  0.5981],
        [ 0.2679, -0.8468],
        [ 2.5389, -2.5702],
        [ 0.7101,  3.0403],
        [-2.4325,  2.3763],
        [-3.4160, -0.8406]], device='mps:0')), ('bias', tensor([-4.5872, -4.8932, -5.0073,  0.0766, -4.6298, -0.7480, -4.9375, -3.8897,
        -4.6235, -5.0654], device='mps:0'))])
OrderedDict([('weight', tensor([[7.9904, 8.8054, 9.0041, 0.3580, 7.2854, 0.8279, 9.1776, 5.8993, 7.5473,
         8.8196]], device='mps:0')), ('bias', tensor([0.4242], device='mps:0'))])
OrderedDict()


...

100%|██████████| 699/699 [00:10<00:00, 63.68it/s] 


Training Loss: 3.5605872514762935 | Validation Loss: 3.2111009803902744
====Epoch 5====
OrderedDict([('weight', tensor([[-1.4898, -2.8513],
        [ 3.1633, -0.3367],
        [ 2.9785,  1.4305],
        [ 0.0462, -0.0497],
        [-3.0259,  0.5626],
        [ 0.3756, -1.5757],
        [ 2.3583, -2.2288],
        [ 0.9150,  3.0491],
        [-2.3172,  2.2750],
        [-3.1703, -0.8522]], device='mps:0')), ('bias', tensor([-5.3382, -5.3278, -5.4578,  0.0690, -5.2872, -1.9296, -5.4553, -4.8805,
        -5.1205, -5.3997], device='mps:0'))])
OrderedDict([('weight', tensor([[10.9655, 11.9600, 12.1980,  0.4096, 10.5864,  2.2617, 12.1127,  9.0684,
         10.6732, 12.1294]], device='mps:0')), ('bias', tensor([0.5566], device='mps:0'))])
OrderedDict()


100%|██████████| 699/699 [00:10<00:00, 64.94it/s] 


Training Loss: 3.0583411743371443 | Validation Loss: 2.8167870870003333
====Epoch 6====
OrderedDict([('weight', tensor([[-1.4374, -2.7854],
        [ 3.0268, -0.3222],
        [ 2.8573,  1.3745],
        [ 0.0323, -0.0227],
        [-2.9077,  0.5413],
        [ 0.3486, -1.6997],
        [ 2.3018, -2.1087],
        [ 0.8737,  2.9778],
        [-2.2318,  2.1591],
        [-3.0311, -0.8151]], device='mps:0')), ('bias', tensor([-5.3604, -5.2002, -5.3403,  0.0645, -5.2412, -2.2477, -5.3891, -4.9119,
        -5.0141, -5.2509], device='mps:0'))])
OrderedDict([('weight', tensor([[11.4844, 12.5000, 12.7605,  0.3874, 11.1722,  2.6464, 12.6085,  9.6274,
         11.2150, 12.7094]], device='mps:0')), ('bias', tensor([0.5207], device='mps:0'))])
OrderedDict()


 33%|███▎      | 228/699 [00:02<00:04, 97.80it/s] 